In [99]:
# library loads and function definitions

import numpy as np
import random as rng

# generate_nodes: input n -> output list like [0,1,...,n]
def generate_nodes(n):
    return list(range(0,n))

def generate_edges(nodes):
    edges = []
    for i in nodes:
        for j in nodes:
            if i < j:
                edges.append(str(i)+str(j))
                edges.append(str(j)+str(i))
    return edges

# generate_distances: input a list of edges (both directions pairwise, "01" followed by "10" or
# vice versa) and integers min_dist and max_dist -> 
# output dictionary with inputs as keys, and random values uniformly generated between min_dist and 
# max_dist assigned to each key, then assign the same value to each key where the string is reversed
def generate_distances(edges, min_dist, max_dist):
    graph = {}
    trimmed_edges = edges[0::2]
    for i in trimmed_edges:
        graph[i] = rng.uniform(min_dist,max_dist)
        graph[i[::-1]] = graph[i]
    
    return graph

def generate_probabilities(edges,)

def assign_labels(nodes):
    places = {}
    places["colony"] = nodes[rng.randint(0,nodes[-1])]
    places["food"] = nodes[rng.randint(0,nodes[-1])]
    
    while places["food"] == places["colony"]:
        places["food"] = nodes[rng.randint(0,nodes[-1])]
        
    return places



In [116]:
nodes = generate_nodes(6)
places = assign_labels(nodes)
edges = generate_edges(nodes)
distances = generate_distances(edges, 0, 25)

print(places)
# check if value for "xy" == "yx" for any valid x and y
# print(graph["01"] == graph["10"])

{'colony': 5, 'food': 2}
